# ALEIA - Cas NLP

### Authors

- Célian Raimbault
- Joséphine Bernard
- Mathieu Latournerie
- Quentin Fisch

### Imports

In [1]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu

In [2]:
import pandas as pd

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import DataFrameLoader

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import Chroma
from langchain import hub
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
import os

load_dotenv()

True

### Data exploration

In [20]:
df = pd.read_csv('data.csv', sep=';')
df.head()

,ID_AIDE,Collectivité,Code SIREN de la collectivité,Référence administrative,Nom de l'aide / de la démarche,Titre alternatif de l'aide,Chapô (HTML),Chapô (TXT),Date,Thème,...,Infos pratiques - Contact (HTML),Infos pratiques - Contact (TXT),Engagements du bénéficiaire,Date ouverture,Date clôture,Délibération cadre,Lien vers le portail iledefrance.fr,Démarche en ligne,DATEDEBUTFUTURCAMPAGNE,qui
0,HJlWyZsxp,Région Île-de-France,237500079,646,RÊV’ELLES : Programme « RVL Découverte »,Olympe - Rêv’Elles : Programme « RVL Découverte »,"<p style=""margin-bottom:0cm;text-align:left;"">...",Ce programme de la Région a pour ambition de f...,2023-10-17T21:30:05+02:00,Éducation & recherche : Lycée,...,"<p><span style=""font-family:CIDFont+F2;"">Marie...","Marie Castanet, responsable des programmes et ...",NaN,2023-09-01T00:00:00+02:00,2024-06-30T00:00:00+02:00,NaN,https://www.iledefrance.fr/aides-et-appels-a-p...,False,NaN,Établissement d'enseignement secondaire|||Part...
1,rkZe_5clT,Région Île-de-France,237500079,557,Appel à projets Actions de formation en situat...,Appel à projets Actions de formation en situat...,"<p style=""margin-bottom:0cm;line-height:normal...",La Région soutient des actions de formation in...,2023-10-04T21:30:05+02:00,Emploi & formation : Formation professionnelle...,...,"<p><a href=""mailto:aap.afest@iledefrance.fr"" t...",aap.afest@iledefrance.fr Les questions sur AA...,Charte des valeurs de la République et de la l...,2022-07-07T00:00:00+02:00,2022-09-30T23:59:00+02:00,NaN,https://www.iledefrance.fr/aides-et-appels-a-p...,False,NaN,"Établissement ou organismes de formation (OF, ..."
2,r1gaxD2xa,Région Île-de-France,237500079,114,Aide aux projets œuvrant à la valorisation des...,NaN,Actions visant à valoriser et mieux faire conn...,Actions visant à valoriser et mieux faire conn...,2023-10-05T21:30:10+02:00,Culture : Patrimoine,...,"Chargée de mission pour les départements 75, 7...","Chargée de mission pour les départements 75, 7...","Mesure ""Trouvez un stage"",Charte des valeurs d...",NaN,NaN,CR 2017-084 modifié par CR 2020-015,https://www.iledefrance.fr/aides-et-appels-a-p...,False,NaN,Professionnel - Patrimoine|||Professionnel - C...
3,rJWfsdW-T,Région Île-de-France,237500079,23,"Parentalité, ouverture socio-culturelle et sco...","Parentalité, ouverture socio-culturelle et sco...","Dans le cadre de son programme<a href="" https:...",Dans le cadre de son programme Région solidair...,2023-10-09T21:30:37+02:00,Santé & Social : Action sociale,...,"<a href=""mailto:solidarites@iledefrance.fr"">so...",solidarites@iledefrance.fr,"Mesure ""Trouvez un stage"",Charte des valeurs d...",2023-03-20T00:00:00+01:00,2023-05-09T00:00:00+02:00,CP 2018-63,https://www.iledefrance.fr/aides-et-appels-a-p...,False,NaN,Association - Régie par la loi de 1901
4,ByiJ6L2g6,Région Île-de-France,237500079,112,Soutien à la restauration et à l’aménagement d...,NaN,La Région contribue à préserver et favoriser l...,La Région contribue à préserver et favoriser l...,2023-10-24T17:00:05+02:00,Culture : Patrimoine,...,<p>Chargée de mission pour les départements 75...,"Chargée de mission pour les départements 75, 7...","Mesure ""Trouvez un stage"",Charte des valeurs d...",NaN,NaN,CR 2017-084 modifié par CR 2020-015,https://www.iledefrance.fr/aides-et-appels-a-p...,False,NaN,Association - Fondation|||Association - ONG|||...


In [21]:
columns_to_drop = ["ID_AIDE", "Collectivité", "Code SIREN de la collectivité", "Référence administrative", "Chapô (HTML)", "Pour quel type de projet ?", "Précisions sur le public bénéficiaire (HTML)", "Quelle est la nature de l'aide ?", "Quelles démarches ?", "À noter", "Documents informatifs", "Documents nécessaires", "Infos pratiques - Contact (HTML)", "Date", "Délibération cadre", "DATEDEBUTFUTURCAMPAGNE"]
df = df.drop(columns=columns_to_drop)
df.head()

,Nom de l'aide / de la démarche,Titre alternatif de l'aide,Chapô (TXT),Thème,Mots clés,Pour quel type de projet ? (TXT),Qui peut en bénéficier ?,Précisions sur le public bénéficiaire (TXT),Quelle est la nature de l'aide ? (TXT),Quelles démarches ? (TXT),À noter (TXT),Infos pratiques - Contact (TXT),Engagements du bénéficiaire,Date ouverture,Date clôture,Lien vers le portail iledefrance.fr,Démarche en ligne,qui
0,RÊV’ELLES : Programme « RVL Découverte »,Olympe - Rêv’Elles : Programme « RVL Découverte »,Ce programme de la Région a pour ambition de f...,Éducation & recherche : Lycée,Egalité femmes/hommes,RVL Découverte fait découvrir les différents m...,"Établissement d'enseignement secondaire,Partic...",Les jeunes femmes des lycées francilien.,Action gratuite pour les lycées franciliens.,"Pour bénéficier de ce programme, s’inscrire di...",NaN,"Marie Castanet, responsable des programmes et ...",NaN,2023-09-01T00:00:00+02:00,2024-06-30T00:00:00+02:00,https://www.iledefrance.fr/aides-et-appels-a-p...,False,Établissement d'enseignement secondaire|||Part...
1,Appel à projets Actions de formation en situat...,Appel à projets Actions de formation en situat...,La Région soutient des actions de formation in...,Emploi & formation : Formation professionnelle...,Emploi,Les projets soutenus sont des actions de forma...,"Établissement ou organismes de formation (OF, ...","Les porteurs éligibles sont : Les entreprises,...",Le taux d'intervention maximum sur le projet s...,Les dossiers devront être déposés sur mesdemar...,NaN,aap.afest@iledefrance.fr Les questions sur AA...,Charte des valeurs de la République et de la l...,2022-07-07T00:00:00+02:00,2022-09-30T23:59:00+02:00,https://www.iledefrance.fr/aides-et-appels-a-p...,False,"Établissement ou organismes de formation (OF, ..."
2,Aide aux projets œuvrant à la valorisation des...,NaN,Actions visant à valoriser et mieux faire conn...,Culture : Patrimoine,"Patrimoine,Attractivité",Actions de valorisation (expos physiques ou vi...,"Professionnel - Patrimoine,Professionnel - Cul...",Sont éligibles les propriétaires publics et pr...,La subvention est calculée par application d’u...,Les demandes d’aide devront être déposées sur ...,Ouverture permanente,"Chargée de mission pour les départements 75, 7...","Mesure ""Trouvez un stage"",Charte des valeurs d...",NaN,NaN,https://www.iledefrance.fr/aides-et-appels-a-p...,False,Professionnel - Patrimoine|||Professionnel - C...
3,"Parentalité, ouverture socio-culturelle et sco...","Parentalité, ouverture socio-culturelle et sco...",Dans le cadre de son programme Région solidair...,Santé & Social : Action sociale,Région solidaire,Les projets doivent correspondre à l’une ou pl...,Association - Régie par la loi de 1901,Associations de loi 1901 : l'association doit ...,Subvention en fonctionnement : la subvention r...,Dépôt du dossier de candidature sur la platefo...,Le bénéficiaire de toute subvention de la Régi...,solidarites@iledefrance.fr,"Mesure ""Trouvez un stage"",Charte des valeurs d...",2023-03-20T00:00:00+01:00,2023-05-09T00:00:00+02:00,https://www.iledefrance.fr/aides-et-appels-a-p...,False,Association - Régie par la loi de 1901
4,Soutien à la restauration et à l’aménagement d...,NaN,La Région contribue à préserver et favoriser l...,Culture : Patrimoine,"Patrimoine,Attractivité","Restauration du bâtiment (clos et couvert), Me...","Association - Fondation,Association - ONG,Asso...",Dispositif ouvert aux propriétaires publics ou...,La subvention jusqu'à 30 % des dépenses éligib...,Les demandes d’aide devront être déposées sur ...,Ouverture permanente,"Chargée de mission pour les départements 75, 7...","Mesure ""Trouvez un stage"",Charte des valeurs d...",NaN,NaN,https://www.iledefrance.fr/aides-et-appels-a-p...,False,Association - Fondation|||Association - ONG|||...


In [6]:
df['text'] = "Nom de l'aide: " + df["Nom de l'aide / de la démarche"] + "\n" + "Chapo: " + df["Chapô (TXT)"] + "\n" + "Mots clés: " + df["Mots clés"]
df["text"]

0      Nom de l'aide: RÊV’ELLES : Programme « RVL Déc...
1      Nom de l'aide: Appel à projets Actions de form...
2      Nom de l'aide: Aide aux projets œuvrant à la v...
3      Nom de l'aide: Parentalité, ouverture socio-cu...
4      Nom de l'aide: Soutien à la restauration et à ...
                             ...                        
322    Nom de l'aide: CONVENTIONS DE DEVELOPPEMENT CU...
323    Nom de l'aide: PM'up Souveraineté Transition é...
324    Nom de l'aide: Fonds d'aide à la création de j...
325    Nom de l'aide: Soutien aux projets associatifs...
326    Nom de l'aide: Soutien aux commerces ruraux : ...
Name: text, Length: 327, dtype: object

In [7]:
df_reduced = df[["text", "qui", "Thème"]]
df_reduced

,text,qui,Thème
0,Nom de l'aide: RÊV’ELLES : Programme « RVL Déc...,Établissement d'enseignement secondaire|||Part...,Éducation & recherche : Lycée
1,Nom de l'aide: Appel à projets Actions de form...,"Établissement ou organismes de formation (OF, ...",Emploi & formation : Formation professionnelle...
2,Nom de l'aide: Aide aux projets œuvrant à la v...,Professionnel - Patrimoine|||Professionnel - C...,Culture : Patrimoine
3,"Nom de l'aide: Parentalité, ouverture socio-cu...",Association - Régie par la loi de 1901,Santé & Social : Action sociale
4,Nom de l'aide: Soutien à la restauration et à ...,Association - Fondation|||Association - ONG|||...,Culture : Patrimoine
...,...,...,...
322,Nom de l'aide: CONVENTIONS DE DEVELOPPEMENT CU...,Professionnel - Créateur d'entreprise|||Profes...,Culture : Arts plastiques; numériques et urbains
323,Nom de l'aide: PM'up Souveraineté Transition é...,Professionnel - TPE < 10|||Professionnel - PME...,"Territoire : SDRIF Environnemental,Économie : ..."
324,Nom de l'aide: Fonds d'aide à la création de j...,Professionnel - Créateur d'entreprise|||Profes...,Culture : Jeux vidéo
325,Nom de l'aide: Soutien aux projets associatifs...,Association - Régie par la loi de 1901,Santé & Social : Action sociale


In [8]:
loader = DataFrameLoader(df, page_content_column="Nom de l'aide / de la démarche")
data = loader.load()
data[:10]

[Document(page_content='RÊV’ELLES : Programme « RVL Découverte »', metadata={"Titre alternatif de l'aide": 'Olympe - Rêv’Elles : Programme « RVL Découverte »', 'Chapô (TXT)': 'Ce programme de la Région a pour ambition de faire découvrir aux jeunes filles un grand nombre de secteurs et de métiers avec des objectifs pédagogiques d’ouverture du champ des possibles, d’affranchissement des déterminismes et de pouvoir d’agir.', 'Thème': 'Éducation & recherche : Lycée', 'Mots clés': 'Egalité femmes/hommes', 'Pour quel type de projet ? (TXT)': "RVL Découverte fait découvrir les différents métiers et secteurs d’activités aux jeunes filles et les accompagne à dépasser stéréotypes de genre, difficultés de représentation des métiers, biais d’orientation et autocensure.Sensibilisation sur la déconstruction des croyances limitantes auprès d’établissements scolaires ou de structures jeunesse.Programme RVL Ton Potentiel : parcours intensif de 5 jours puis suivi de 5 mois avec coachings collectifs et i

### Méthode 1 - Vector database simple

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(data)
docs[:2]

[Document(page_content='RÊV’ELLES : Programme « RVL Découverte »', metadata={"Titre alternatif de l'aide": 'Olympe - Rêv’Elles : Programme « RVL Découverte »', 'Chapô (TXT)': 'Ce programme de la Région a pour ambition de faire découvrir aux jeunes filles un grand nombre de secteurs et de métiers avec des objectifs pédagogiques d’ouverture du champ des possibles, d’affranchissement des déterminismes et de pouvoir d’agir.', 'Thème': 'Éducation & recherche : Lycée', 'Mots clés': 'Egalité femmes/hommes', 'Pour quel type de projet ? (TXT)': "RVL Découverte fait découvrir les différents métiers et secteurs d’activités aux jeunes filles et les accompagne à dépasser stéréotypes de genre, difficultés de représentation des métiers, biais d’orientation et autocensure.Sensibilisation sur la déconstruction des croyances limitantes auprès d’établissements scolaires ou de structures jeunesse.Programme RVL Ton Potentiel : parcours intensif de 5 jours puis suivi de 5 mois avec coachings collectifs et i

In [10]:
# use distilbert to embed the text
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/Users/quentinfisch/Documents/EPITA/ING3/SCIA/S9/ALEIA/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
db = FAISS.from_documents(docs, embeddings)

In [12]:
question = "Mon logement tombe en ruines. Il faudrait le rénover. Est-ce qu'il y a une aide régionale pour ça?"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

Renouvellement des chauffages au fuel, au bois ou au charbon


### Méthode 2: Langchain + LLM (GPT-3.5-turbo)

In [14]:
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

In [66]:
loader = DataFrameLoader(df, page_content_column="Nom de l'aide / de la démarche")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)
docs[:2]

[Document(page_content='RÊV’ELLES : Programme « RVL Découverte »', metadata={"Titre alternatif de l'aide": 'Olympe - Rêv’Elles : Programme « RVL Découverte »', 'Chapô (TXT)': 'Ce programme de la Région a pour ambition de faire découvrir aux jeunes filles un grand nombre de secteurs et de métiers avec des objectifs pédagogiques d’ouverture du champ des possibles, d’affranchissement des déterminismes et de pouvoir d’agir.', 'Thème': 'Éducation & recherche : Lycée', 'Mots clés': 'Egalité femmes/hommes', 'Pour quel type de projet ? (TXT)': "RVL Découverte fait découvrir les différents métiers et secteurs d’activités aux jeunes filles et les accompagne à dépasser stéréotypes de genre, difficultés de représentation des métiers, biais d’orientation et autocensure.Sensibilisation sur la déconstruction des croyances limitantes auprès d’établissements scolaires ou de structures jeunesse.Programme RVL Ton Potentiel : parcours intensif de 5 jours puis suivi de 5 mois avec coachings collectifs et i

In [70]:
embeddings = OpenAIEmbeddings()
vector_store = Chroma.from_documents(docs, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 4}, search_type="similarity")

In [86]:
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

template = """
Voici les différentes aides que le client peut obtenir:
{context}

Tu dois répondre en détaillant les 2 aides qui te semblent les plus pertinentes pour le client. Pour chaque aide, indique le nom de l'aide tel qu'il est écrit dans le document, puis une description globale de l'aide.

La question posée par le client est la suivante:
{question}
"""
prompt = ChatPromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#### Test 1: recherche d'aide à partir d'une question générique

In [87]:
print(rag_chain.invoke("Je veux passer à la voiture électrique"))

Les deux aides les plus pertinentes pour le client qui souhaite passer à la voiture électrique sont :

1. Plan route de demain : Électromobilité
Cette aide consiste en un plan d'action complet pour faciliter la transition vers l'électromobilité. Il comprend des conseils personnalisés sur le choix du véhicule électrique, l'installation de bornes de recharge à domicile, les avantages fiscaux liés à l'achat d'un véhicule électrique, ainsi que des informations sur les infrastructures de recharge publiques disponibles. Le plan route de demain : Électromobilité vise à accompagner le client à chaque étape de sa transition vers la voiture électrique, en lui fournissant toutes les informations et les ressources nécessaires.

2. Transformation de véhicules thermiques en véhicules propres
Cette aide permet au client de transformer son véhicule thermique en un véhicule propre, en le convertissant à l'électromobilité. Elle offre des subventions pour la conversion de moteurs thermiques en moteurs él

In [18]:
print(rag_chain.invoke("Je veux acheter un food truck électrique"))

Les deux aides les plus pertinentes pour le client qui souhaite acheter un food truck électrique sont :

1. Commerce ou artisanat ambulant : acquisition de véhicules propres
Cette aide est spécifiquement destinée aux commerçants ou artisans ambulants qui souhaitent acquérir un véhicule propre, tel qu'un food truck électrique. Elle peut prendre la forme d'une subvention ou d'une aide financière pour faciliter l'achat du véhicule. Cette aide est particulièrement pertinente car elle est spécifiquement conçue pour soutenir les professionnels qui souhaitent investir dans des véhicules propres, ce qui correspond aux besoins du client.

2. Acquisition de véhicules propres par les professionnels franciliens
Cette aide est destinée aux professionnels de la région francilienne qui souhaitent acquérir des véhicules propres, y compris les food trucks électriques. Elle peut prendre la forme d'une subvention ou d'une aide financière pour réduire le coût d'achat du véhicule. Cette aide est pertinente

Les résultats sont pertinents et permettent de créer un chatbot qui indique à l'utilisateur les différentes aides auxquelles il peut prétendre en fonction de sa situation.

#### Test 2: demande d'informations sur une aide

In [27]:
df[df["Nom de l\'aide / de la démarche"] == "Plan route de demain : Électromobilité"]["Quelles démarches ? (TXT)"].item()

'Déposez sur mesdemarches.iledefrance.fr votre dossier de demande de subvention présentant le projet de manière précise et comportant toutes les pièces permettant l’instruction du dossier, réputé complet, par les services de la Région. Après examen du dossier, la demande de subvention sera soumise à la Commission permanente régionale pour délibération.  Le versement de la subvention est subordonné à la signature préalable d’une convention.'

In [19]:
print(rag_chain.invoke("Concernant l'aide 'Plan route de demain : Électromobilité', quelles sont les démarches à faire ?"))

Pour bénéficier de l'aide "Plan route de demain : Électromobilité", le client devra suivre les démarches suivantes :

1. Renseignement sur les critères d'éligibilité : Le client devra se renseigner sur les critères d'éligibilité spécifiques à cette aide. Il peut s'agir de critères liés au type de véhicule électrique, à la localisation géographique, aux revenus, etc. Il est important de vérifier si le client remplit toutes les conditions requises pour bénéficier de cette aide.

2. Collecte des documents nécessaires : Le client devra rassembler tous les documents requis pour constituer son dossier de demande d'aide. Cela peut inclure des pièces d'identité, des justificatifs de domicile, des factures d'achat ou de location du véhicule électrique, des preuves de revenus, etc. Il est essentiel de s'assurer d'avoir tous les documents demandés pour éviter tout retard dans le traitement de la demande.

3. Remplissage du formulaire de demande : Le client devra remplir le formulaire de demande d

Concernant la recherche d'informations plus détaillée dans pour une aide, la réponse est pertinente mais ne contient pas tous les éléments indiqué dans les données initiales.

In [25]:
df_questions = pd.read_excel("Aides_Questions.xlsx")
df_questions.head()

,Id_AideQuestion_Num,ID,Questions,titre
0,51,635,Je voudrais pouvoir moderniser mon exploitatio...,FEADER - Soutien aux investissements agricoles...
1,158,595,j'en ai marre de paris et sa petite couronne. ...,FEADER - Aide à l'installation du jeune agricu...
2,218,80,Nous cherchons à créer un réseau de chaleur ba...,Développement d'unités de méthanisation (études)
3,259,364,Je suis un producteur de laitue en difficulté ...,Soutien à l’agriculture dans les zones périurb...
4,279,79,Quels sont les aides pour la methanisation ?,Développement d’unités de méthanisation


In [29]:
df_questions["titre"][0]

'FEADER - Soutien aux investissements agricoles - Modernisation des exploitations (73.01.01)'

In [42]:
print(rag_chain.invoke(df_questions["Questions"][0]))

Les deux aides les plus pertinentes pour le client sont :

1. Aide à la transformation agroalimentaire francilienne :
Cette aide vise à soutenir les agriculteurs dans la modernisation de leurs exploitations en favorisant la transformation de leurs produits agricoles. Elle permet notamment d'accompagner financièrement les investissements liés à l'amélioration des équipements de transformation, à l'innovation technologique, à la mise en conformité sanitaire, à la diversification des produits, etc. Cette aide est spécifiquement dédiée aux agriculteurs franciliens et vise à renforcer la compétitivité et la valorisation des produits agricoles de la région.

2. FEADER - Soutien aux investissements agricoles - Modernisation des exploitations (73.01.01) :
Cette aide du Fonds européen agricole pour le développement rural (FEADER) vise à soutenir les investissements agricoles visant à moderniser les exploitations. Elle permet de financer des projets tels que l'acquisition de matériel agricole pe

### Évaluation

Pour évaluer le modèle, on utilise le dataset de questions / titre fourni. On teste si le titre attendu pour chaque question est contenu dans la réponse du chatbot.

In [89]:
import random

def test_random_questions(df_questions, rag_chain, N=10):
    valid = N
    for i in range(N):
        index = random.randint(0, len(df_questions) - 1)
        question = df_questions["Questions"][index]
        answer = rag_chain.invoke(question)
        if df_questions["titre"][index].lower() not in answer.lower():
            valid -= 1
            print("Question: " + question)
            print("Answer: " + answer)
            print("Title: " + df_questions["titre"][index])
            print("--------------------------------------------------------")
    print("Accuracy: " + str(valid/N * 100) + "%")

In [90]:
test_random_questions(df_questions, rag_chain, N=5)

Question: J'ai une petite entreprise innovante dans le milieu du vin.
Answer: Les deux aides les plus pertinentes pour le client sont les suivantes:

1. Rencontres Entreprises Enseignants "Mon réseau pro":
Cette aide consiste en des rencontres organisées entre les entreprises et les enseignants, dans le but de favoriser les échanges et les collaborations. Ces rencontres permettent aux entreprises innovantes, comme celle du client dans le milieu du vin, de rencontrer des enseignants spécialisés dans leur domaine d'activité. Cela peut être bénéfique pour le client car il pourra bénéficier de l'expertise des enseignants et établir des partenariats pour développer son entreprise.

2. Innov'up Expérimentation Silver Economie / Bien vieillir:
Cette aide est spécifiquement destinée aux entreprises innovantes dans le domaine de la Silver Economie et du bien vieillir. Bien que le client soit dans le milieu du vin, il peut également bénéficier de cette aide s'il propose des produits ou services 

Le modèle se rapporche très souvent de la réponse attendue, sans pour autant parvenir à écrire le titre littéral. L'accuracy retournée est donc biaisée, malgré que le modèle soit objectivement bon.